<a href="https://colab.research.google.com/github/MAR123EREE/Laboratorio_02/blob/main/Lab_04_de_reg_log_onevsall_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# utilizado para la manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np

# Libreria para graficos
from matplotlib import pyplot

# Modulo de optimizacion en scipy
from scipy import optimize

# modulo para cargar archivos en formato MATLAB
# from scipy.io import loadmat

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

In [6]:
import pandas as pd
# La entrada es de 784 elemento contando con x0
input_layer_size  = 784
# 10 etiquetas, de 1 a 10 (tomar en cuenta que se asigna "0" a la etiqueta 10), en este caso son 10 etiquetas
num_labels = 10
#  datos de entrenamiento almacenados en los arreglos X, y
data = pd.read_csv('/content/drive/MyDrive/Data_sets/Lab_04/mnist_train.csv', delimiter=',')
X = data.iloc[:, :1]
#todo
y = data.iloc[:, 0]
m = y.size

In [7]:
print(X.iloc[0,:])
print(y)

label    5
Name: 0, dtype: int64
0        5
1        0
2        4
3        1
4        9
        ..
59995    8
59996    3
59997    5
59998    6
59999    8
Name: label, Length: 60000, dtype: int64


In [16]:
# Funcion train_test_split, que recibe X y y, test_size es el tamaño de prueba
#Aqui depende de cuanta parte del cod debe ser de prueba y entenamiento
!pip install scikit-learn # install scikit-learn if you haven't already
from sklearn.model_selection import train_test_split # import the train_test_split function
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

Imprimiendo la división del dataset para entrenamiento y prueba

In [17]:
print("Datos entrenamiento> ")
print(X_train)
print('-'*40)
print(y_train)

Datos entrenamiento> 
          label
20379 -0.157111
53032 -0.157111
27005  0.881222
30510  1.227333
508   -1.195444
...         ...
55169 -1.195444
49861 -0.157111
27063 -1.195444
8366   1.227333
17530  1.227333

[48000 rows x 1 columns]
----------------------------------------
20379    4
53032    4
27005    7
30510    8
508      1
        ..
55169    1
49861    4
27063    1
8366     8
17530    8
Name: label, Length: 48000, dtype: int64


In [18]:
print("Datos prueba> ")
print(X_test)
print('-'*40)
print(y_test)

Datos prueba> 
          label
41382 -0.849333
20883  0.881222
13621  0.535111
41823 -1.195444
33839  1.227333
...         ...
34502 -1.541555
730    0.881222
17814  0.535111
37767 -0.157111
39655 -0.849333

[12000 rows x 1 columns]
----------------------------------------
41382    2
20883    7
13621    6
41823    1
33839    8
        ..
34502    0
730      7
17814    6
37767    4
39655    2
Name: label, Length: 12000, dtype: int64


In [19]:
#Normalizacion de datos, desviacion estandar y es un código que se apróxima
def  featureNormalize(X):
    X_norm = X_train.copy()
    mu = np.zeros(X_train.shape[1])
    sigma = np.zeros(X_train.shape[1])

    mu = np.mean(X_train, axis = 0)
    sigma = np.std(X_train, axis = 0)
    X_norm = (X_train - mu) / sigma

    return X_norm, mu, sigma

In [23]:
# llama featureNormalize con los datos cargados
X_norm, mu, sigma = featureNormalize(X_train)

In [52]:
print(X_norm.iloc[0,:]) # Usando .iloc for integrar-based indexando en pandas
print(y_train)

label   -0.84255
Name: 41382, dtype: float64
20379    4
53032    4
27005    7
30510    8
508      1
        ..
55169    1
49861    4
27063    1
8366     8
17530    8
Name: label, Length: 48000, dtype: int64


In [11]:
# Configurar la matriz adecuadamente, y agregar una columna de unos que corresponde al termino de intercepción.
m, n = X.shape
X = X_norm

In [26]:
# Selecciona aleatoriamente 100 puntos de datos para mostrar
rand_indices = np.random.choice(m, 100, replace=False)
sel = X.iloc[rand_indices, :]
#displayData(sel)

In [27]:
def sigmoid(z):
    """
    Calcula la sigmoide de z.
    """
    return 1.0 / (1.0 + np.exp(-z))

In [28]:
# Prueba la implementacion de la funcion sigmoid
z = 0
g = sigmoid(z)

print('g(', z, ') = ', g)

g( 0 ) =  0.5


In [43]:
#Lamnda se usa y es un valor pequeño que va multiplicar ciertos valores
def lrCostFunction(theta, X_train, y_train, lambda_):
    """
    Calcula el costo de usar theta como parámetro para la regresión logística regularizada y
    el gradiente del costo w.r.t. a los parámetros.

    Parametros
    ----------
    theta : array_like
        Parametro theta de la regresion logistica. Vector de la forma(shape) (n, ). n es el numero de caracteristicas
        incluida la intercepcion

    X : array_like
        Dataset con la forma(shape) (m x n). m es el numero de ejemplos, y n es el numero de
        caracteristicas (incluida la intercepcion).

    y : array_like
        El conjunto de etiquetas. Un vector con la forma (shape) (m, ). m es el numero de ejemplos

    lambda_ : float
        Parametro de regularización.

    Devuelve
    -------
    J : float
        El valor calculado para la funcion de costo regularizada.

    grad : array_like
        Un vector de la forma (shape) (n, ) que es el gradiente de la
        función de costo con respecto a theta, en los valores actuales de theta..
    """
#     alpha = 0.003
#     theta = theta.copy()
    # Inicializa algunos valores utiles
    m = y_train.size

    # convierte las etiquetas a valores enteros si son boleanos
    if y_train.dtype == bool:
        y_train = y_train.astype(int)
#esta grad le estamos pasando la cantidad de titas
    J = 0
    grad = np.zeros(theta.shape)

    h = sigmoid(X_train.dot(theta.T))

    temp = theta
    temp[0] = 0

#     J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h)))   el de + es el regulador, eso evitsrs que hay un sobre ajuste
    J = (1 / m) * np.sum(-y_train.dot(np.log(h)) - (1 - y_train).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))

    # Remplazar X con X_train
    grad = (1 / m) * (h - y_train).dot(X_train)
#     theta = theta - (alpha / m) * (h - y).dot(X)
    grad = grad + (lambda_ / m) * temp

    return J, grad
#    return J, theta

<a id="section2"></a>
### 1.4 Clasificacion One-vs-all
En esta parte del ejercicio, se implementará la clasificación de uno contra todos mediante el entrenamiento de múltiples clasificadores de regresión logística regularizados, uno para cada una de las clases $K$ en nuestro conjunto de datos. En el conjunto de datos de dígitos escritos a mano, $K = 10$, pero su código debería funcionar para cualquier valor de $K$.

El argumento `y` de esta función es un vector de etiquetas de 0 a 9. Al entrenar el clasificador para la clase $k \in \{0, ..., K-1 \} $, querrá un vector K-dimensional de etiquetas $y$, donde $y_j \ in 0, 1$ indica si la instancia de entrenamiento $j ^ {th}$ pertenece a la clase $k$ $(y_j = 1)$, o si pertenece a una clase diferente $(y_j = 0)$.

Además, se utiliza `optimize.minimize` de scipy para este ejercicio.
<a id="oneVsAll"></a>

In [44]:
#one vs all esta esta creando una matriz 10*784 porque hay 10 clasificaciones y valores de tetas
def oneVsAll(X_train, y_train, num_labels, lambda_):
    """
    Trains num_labels logistic regression classifiers and returns
    each of these classifiers in a matrix all_theta, where the i-th
    row of all_theta corresponds to the classifier for label i.

    Parameters
    ----------
    X : array_like
        The input dataset of shape (m x n). m is the number of
        data points, and n is the number of features. Note that we
        do not assume that the intercept term (or bias) is in X, however
        we provide the code below to add the bias term to X.

    y : array_like
        The data labels. A vector of shape (m, ).

    num_labels : int
        Number of possible labels.

    lambda_ : float
        The logistic regularization parameter.

    Returns
    -------
    all_theta : array_like
        The trained parameters for logistic regression for each class.
        This is a matrix of shape (K x n+1) where K is number of classes
        (ie. `numlabels`) and n is number of features without the bias.
    """
    # algunas variables utiles
    m, n = X_train.shape

    all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    X_train = np.concatenate([np.ones((m, 1)), X_train], axis=1)

    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 50}
        # en x estan todos los valores de titas que estaba buscando
        res = optimize.minimize(lrCostFunction,
                                initial_theta,
                                (X_train, (y_train == c), lambda_),
                                jac=True,
                                method='CG',
                                options=options)

        all_theta[c] = res.x

    return all_theta

In [41]:
lambda_ = 0.1
all_theta = oneVsAll(X_train, y_train, num_labels, lambda_)
#me esta devolviendo el tamaño
print(all_theta.shape)

(10, 2)


In [42]:
print(all_theta)

[[-49.29873705 -35.96675143]
 [ -3.79931268  -2.48756209]
 [ -2.71547303  -1.17492017]
 [ -2.31186819  -0.60258196]
 [ -2.2291234   -0.17750648]
 [ -2.32821283   0.20698051]
 [ -2.3659796    0.63072216]
 [ -2.68503595   1.21240655]
 [ -3.95651236   2.40767473]
 [-49.61780977  35.44274498]]


In [45]:
def predictOneVsAll(all_theta, X_train):
    """
    Devuelve un vector de predicciones para cada ejemplo en la matriz X.
    Tenga en cuenta que X contiene los ejemplos en filas.
    all_theta es una matriz donde la i-ésima fila es un vector theta de regresión logística entrenada para la i-ésima clase.
    Debe establecer p en un vector de valores de 0..K-1 (por ejemplo, p = [0, 2, 0, 1]
    predice clases 0, 2, 0, 1 para 4 ejemplos).

    Parametros
    ----------
    all_theta : array_like
        The trained parameters for logistic regression for each class.
        This is a matrix of shape (K x n+1) where K is number of classes
        and n is number of features without the bias.

    X : array_like
        Data points to predict their labels. This is a matrix of shape
        (m x n) where m is number of data points to predict, and n is number
        of features without the bias term. Note we add the bias term for X in
        this function.

    Devuelve
    -------
    p : array_like
        The predictions for each data point in X. This is a vector of shape (m, ).
    """
  #numero de valores de m
    m = X_train.shape[0];
    #numero de etiquetas num lebel,
    num_labels = all_theta.shape[0]
    #estamos crando p para la "y" predicha,
    p = np.zeros(m)

    # Add ones to the X data matrix
    X_train = np.concatenate([np.ones((m, 1)), X_train], axis=1)
    #es el q va devolver el que tiene mayor probabilidad me refiero argmax
    p = np.argmax(sigmoid(X_train.dot(all_theta.T)), axis = 1)

    return p

Una vez que haya terminado, se llama a la función `predictOneVsAll` usando el valor aprendido de $\theta$. Debería apreciarse que la precisión del conjunto de entrenamiento es de aproximadamente 81.15%

In [48]:
print(X_train.shape)
#si la prediccion es igual a "y"
pred = predictOneVsAll(all_theta, X_train)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y_train) * 100))
XPrueba = X_train.iloc[10:150, :].copy()  # Colocar .iloc para integrar la locación
print(XPrueba.shape)
#p = np.zeros(1)
XPrueba = np.concatenate([np.ones((140, 1)), XPrueba], axis=1)
print(XPrueba.shape)
p = np.argmax(sigmoid(XPrueba.dot(all_theta.T)), axis = 1)
print(p)

# displayData(X[1002:1003, :])
print(y_train[10:150])

(48000, 1)
Precision del conjuto de entrenamiento: 81.15%
(140, 1)
(140, 2)
[6 6 6 3 4 7 7 1 1 6 3 1 4 0 7 4 8 1 3 1 8 0 1 1 0 1 1 6 4 7 8 7 7 4 4 8 3
 4 6 0 6 6 8 0 4 4 1 9 3 6 6 1 6 9 3 1 4 4 3 8 4 8 4 6 7 7 6 1 6 7 8 8 6 1
 7 8 4 3 9 0 3 1 4 6 9 6 8 7 7 0 7 4 1 8 8 1 7 7 1 6 1 6 6 1 1 1 1 3 4 1 4
 8 6 6 4 7 1 8 1 4 6 8 1 1 6 1 9 1 0 6 4 3 0 4 0 6 7 1 6 6]
45753    5
13047    6
6004     5
49729    3
39703    4
        ..
44262    5
41928    7
20461    1
42509    5
44293    5
Name: label, Length: 140, dtype: int64


La otra parte del dataset de prueba

In [49]:
#Normalizacion de datos, con la desviacion estandar y es un código que se apróxima
def  featureNormalize(X):
    X_norm = X_test.copy()
    mu = np.zeros(X_test.shape[1])
    sigma = np.zeros(X_test.shape[1])

    mu = np.mean(X_test, axis = 0)
    sigma = np.std(X_test, axis = 0)
    X_norm = (X_test - mu) / sigma

    return X_norm, mu, sigma

In [50]:
# llama featureNormalize con los datos cargados
X_norm, mu, sigma = featureNormalize(X_test)

In [51]:
print(X_norm.iloc[0,:]) # Usando .iloc for integrar-based indexando en pandas
print(y_test)

label   -0.84255
Name: 41382, dtype: float64
41382    2
20883    7
13621    6
41823    1
33839    8
        ..
34502    0
730      7
17814    6
37767    4
39655    2
Name: label, Length: 12000, dtype: int64


In [53]:
# Configurar la matriz adecuadamente, y agregar una columna de unos que corresponde al termino de intercepción.
m, n = X.shape
X = X_norm

In [56]:
# Selecciona aleatoriamente 100 puntos de datos para mostrar
rand_indices = np.random.choice(X.shape[0], 100, replace=False) # Use the current shape of X
sel = X.iloc[rand_indices, :]
#displayData(sel)

In [57]:
#Lamnda se usa y es un valor pequeño que va multiplicar ciertos valores
def lrCostFunction(theta, X_test, y_test, lambda_):
    """
    Calcula el costo de usar theta como parámetro para la regresión logística regularizada y
    el gradiente del costo w.r.t. a los parámetros.

    Parametros
    ----------
    theta : array_like
        Parametro theta de la regresion logistica. Vector de la forma(shape) (n, ). n es el numero de caracteristicas
        incluida la intercepcion

    X : array_like
        Dataset con la forma(shape) (m x n). m es el numero de ejemplos, y n es el numero de
        caracteristicas (incluida la intercepcion).

    y : array_like
        El conjunto de etiquetas. Un vector con la forma (shape) (m, ). m es el numero de ejemplos

    lambda_ : float
        Parametro de regularización.

    Devuelve
    -------
    J : float
        El valor calculado para la funcion de costo regularizada.

    grad : array_like
        Un vector de la forma (shape) (n, ) que es el gradiente de la
        función de costo con respecto a theta, en los valores actuales de theta..
    """
#     alpha = 0.003
#     theta = theta.copy()
    # Inicializa algunos valores utiles
    m = y_test.size

    # convierte las etiquetas a valores enteros si son boleanos
    if y_test.dtype == bool:
        y_test = y_test.astype(int)
#esta grad le estamos pasando la cantidad de titas
    J = 0
    grad = np.zeros(theta.shape)

    h = sigmoid(X_test.dot(theta.T))

    temp = theta
    temp[0] = 0

#     J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h)))   el de + es el regulador, eso evitsrs que hay un sobre ajuste
    J = (1 / m) * np.sum(-y_test.dot(np.log(h)) - (1 - y_test).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))

    # Remplazar X con X_train
    grad = (1 / m) * (h - y_test).dot(X_test)
#     theta = theta - (alpha / m) * (h - y).dot(X)
    grad = grad + (lambda_ / m) * temp

    return J, grad
#    return J, theta

In [58]:
#one vs all esta esta creando una matriz 10*784 porque hay 10 clasificaciones y valores de tetas
def oneVsAll(X_test, y_test, num_labels, lambda_):
    """
    Trains num_labels logistic regression classifiers and returns
    each of these classifiers in a matrix all_theta, where the i-th
    row of all_theta corresponds to the classifier for label i.

    Parameters
    ----------
    X : array_like
        The input dataset of shape (m x n). m is the number of
        data points, and n is the number of features. Note that we
        do not assume that the intercept term (or bias) is in X, however
        we provide the code below to add the bias term to X.

    y : array_like
        The data labels. A vector of shape (m, ).

    num_labels : int
        Number of possible labels.

    lambda_ : float
        The logistic regularization parameter.

    Returns
    -------
    all_theta : array_like
        The trained parameters for logistic regression for each class.
        This is a matrix of shape (K x n+1) where K is number of classes
        (ie. `numlabels`) and n is number of features without the bias.
    """
    # algunas variables utiles
    m, n = X_test.shape

    all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    X_test = np.concatenate([np.ones((m, 1)), X_test], axis=1)

    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 50}
        # en x estan todos los valores de titas que estaba buscando
        res = optimize.minimize(lrCostFunction,
                                initial_theta,
                                (X_test, (y_test == c), lambda_),
                                jac=True,
                                method='CG',
                                options=options)

        all_theta[c] = res.x

    return all_theta

In [59]:
lambda_ = 0.1
all_theta = oneVsAll(X_test, y_test, num_labels, lambda_)
#me esta devolviendo el tamaño
print(all_theta.shape)

(10, 2)


In [60]:
print(all_theta)

[[-39.57575105 -28.87821592]
 [ -3.80648878  -2.47130502]
 [ -2.6555693   -1.17282916]
 [ -2.31370631  -0.59220682]
 [ -2.28015288  -0.16685606]
 [ -2.32320914   0.21713813]
 [ -2.37955239   0.64130976]
 [ -2.65819818   1.23407476]
 [ -3.98792567   2.45047157]
 [-39.98908385  28.55228736]]


In [63]:
def predictOneVsAll(all_theta, X_test):
    """
    Devuelve un vector de predicciones para cada ejemplo en la matriz X.
    Tenga en cuenta que X contiene los ejemplos en filas.
    all_theta es una matriz donde la i-ésima fila es un vector theta de regresión logística entrenada para la i-ésima clase.
    Debe establecer p en un vector de valores de 0..K-1 (por ejemplo, p = [0, 2, 0, 1]
    predice clases 0, 2, 0, 1 para 4 ejemplos).

    Parametros
    ----------
    all_theta : array_like
        The trained parameters for logistic regression for each class.
        This is a matrix of shape (K x n+1) where K is number of classes
        and n is number of features without the bias.

    X : array_like
        Data points to predict their labels. This is a matrix of shape
        (m x n) where m is number of data points to predict, and n is number
        of features without the bias term. Note we add the bias term for X in
        this function.

    Devuelve
    -------
    p : array_like
        The predictions for each data point in X. This is a vector of shape (m, ).
    """
  #numero de valores de m
    m = X_test.shape[0];
    #numero de etiquetas num lebel,
    num_labels = all_theta.shape[0]
    #estamos crando p para la "y" predicha,
    p = np.zeros(m)

    # Add ones to the X data matrix
    X_test = np.concatenate([np.ones((m, 1)), X_test], axis=1)
    #es el q va devolver el que tiene mayor probabilidad me refiero argmax
    p = np.argmax(sigmoid(X_test.dot(all_theta.T)), axis = 1)

    return p

In [64]:
print(X_test.shape)
#si la prediccion es igual a "y"
pred = predictOneVsAll(all_theta, X_test)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y_test) * 100))
XPrueba = X_test.iloc[10:150, :].copy()  # Colocar .iloc para integrar la locación
print(XPrueba.shape)
#p = np.zeros(1)
XPrueba = np.concatenate([np.ones((140, 1)), XPrueba], axis=1)
print(XPrueba.shape)
p = np.argmax(sigmoid(XPrueba.dot(all_theta.T)), axis = 1)
print(p)

# displayData(X[1002:1003, :])
print(y_test[10:150])

(12000, 1)
Precision del conjuto de entrenamiento: 71.78%
(140, 1)
(140, 2)
[2 9 9 8 2 7 0 7 8 0 9 1 0 3 1 2 2 3 3 8 0 0 1 2 6 2 6 1 0 7 7 2 3 2 3 7 6
 3 7 8 6 3 6 3 9 7 7 3 2 6 9 3 2 9 0 3 2 7 3 7 3 7 8 0 2 7 7 1 9 6 0 3 8 7
 3 1 9 7 1 7 7 2 7 3 2 2 9 7 3 2 3 7 0 7 6 2 2 7 3 1 9 1 9 1 9 9 0 7 3 6 6
 9 9 1 7 8 3 9 2 1 7 8 2 6 1 2 3 9 6 2 7 9 3 6 3 0 7 8 1 9]
32523    2
30698    9
16588    9
35365    8
55220    2
        ..
56       0
35648    7
50342    8
42339    1
23919    9
Name: label, Length: 140, dtype: int64


Hay una diferencia de la precisión del conjubto de entrenamiento; entre train(81.15%) y test(71.78%)